In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ---------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
save_res <- TRUE

In [3]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("CESC", "UCS", "UCEC")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [4]:
matrisome_df <- rutils::load_matrisome_df(matrisome_path) %>%
    dplyr::select(gene_symbol, division, category) %>%
    dplyr::rename(geneID = gene_symbol)
# demg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_demg_list.txt"))
# figo_umsmg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_figo_umsmg_list.txt"))
# survival_umsmg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_survival_umsmg_list.txt"))
# figo_nsmg_list <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_figo_nsmg_list.txt"))

# figo_go_df <- read_tsv(paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_figo_go.tsv"))
# survival_go_df <- read_tsv(paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_survival_go.tsv"))

figo_go <- list()
survival_go <- list()

for (dset_idx in 1:3) {
    figo_go[[projects[dset_idx]]] <- read_tsv(paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_figo_go.tsv")) %>%
        select(type, qval)
    survival_go[[projects[dset_idx]]] <- read_tsv(paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_survival_go.tsv")) %>%
        select(type, qval)
}


-- Column specification ---------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)



-- Column specification ---------------------------------------------------------------------------------------
cols(
  type = col_character(),
  geneIDs = col_character(),
  count = col_double(),
  ratio = col_double(),
  qval = col_double(),
  ont = col_character()
)



-- Column specification ---------------------------------------------------------------------------------------
cols(
  type = col_character(),
  geneIDs = col_character(),
  count = col_double(),
  ratio = col_double(),
  qval = col_double(),
  ont = col_chara

In [5]:
topn_lists <- function(go_dfs, n = 10) {
    topn_intersect <- (go_dfs$CESC %>% top_n(n = n, wt = desc(qval)) %>% pull(type)) %>%
        intersect((go_dfs$UCEC %>% top_n(n = n, wt = desc(qval)) %>% pull(type))) %>%
        intersect((go_dfs$UCS %>% top_n(n = n, wt = desc(qval)) %>% pull(type)))
    topn_union <- (go_dfs$CESC %>% top_n(n = n, wt = desc(qval)) %>% pull(type)) %>%
        union((go_dfs$UCEC %>% top_n(n = n, wt = desc(qval)) %>% pull(type))) %>%
        union((go_dfs$UCS %>% top_n(n = n, wt = desc(qval)) %>% pull(type)))
    list(
        topn_intersect = topn_intersect,
        topn_union = topn_union
    )
}

full_lists <- function(go_dfs) {
    full_intersect <- go_dfs$CESC$type %>%
        intersect(go_dfs$UCEC$type) %>%
        intersect(go_dfs$UCS$type)
    full_union <- go_dfs$CESC$type %>%
        union(go_dfs$UCEC$type) %>%
        union(go_dfs$UCS$type)
    list(
        full_intersect = full_intersect,
        full_union = full_union
    )
}

# FIGO

## Proportion of total shared by all

In [6]:
figo_full_lists <- full_lists(figo_go)
length(figo_full_lists$full_union)
length(figo_full_lists$full_intersect)
length(figo_full_lists$full_intersect) / length(figo_full_lists$full_union)

[1] 908

[1] 183

[1] 0.2015419

## Proportion of top n shared by all

In [11]:
n <- 10
# n <- 20
figo_topn_lists <- topn_lists(figo_go, n = n)
length(figo_topn_lists$topn_union)
length(figo_topn_lists$topn_intersect)
length(figo_topn_lists$topn_intersect) / length(figo_topn_lists$topn_union)

[1] 17

[1] 5

[1] 0.2941176

# Survival

## Proportion of total shared by all

In [8]:
survival_full_lists <- full_lists(survival_go)
length(survival_full_lists$full_union)
length(survival_full_lists$full_intersect)
length(survival_full_lists$full_intersect) / length(survival_full_lists$full_union)

[1] 457

[1] 8

[1] 0.01750547

## Proportion of top n shared by all

In [9]:
n <- 10
survival_topn_lists <- topn_lists(survival_go, n = n)
length(survival_topn_lists$topn_union)
length(survival_topn_lists$topn_intersect)
length(survival_topn_lists$topn_intersect) / length(survival_topn_lists$topn_union)

[1] 25

[1] 1

[1] 0.04